In [4]:
import sqlite3
from sqlite3 import Error
import pandas as pd
 
def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        print(sqlite3.version)
    except Error as e:
        print(e)
 
    return conn
 
def select_all_tasks(conn, question, query_txt):
    """
    Query all rows in the tasks table
    :param conn: the Connection object
    :return:
    """
    cur = conn.cursor()
    
    print(question)
    print()
    print(query_txt)
    
   
    cur.execute(query_txt)
 
    field_names = [i[0] for i in cur.description]
    rows = cur.fetchall()
     
    #print(field_names)    
    #for row in rows:
    #    print(row)
    print()    
    db_table = pd.DataFrame(rows, columns=field_names)    
    print(db_table)
    print()


def main():
    database = "sqlite\db\pythonsqlite.db"
    
    q10 = """Q10. Produce a list of facilities with a total revenue less than 1000.
           The output of facility name and total revenue, sorted by revenue. Remember
           that there's a different cost for guests and members!"""
    
    query_10 = """SELECT sub.facility_name, sub.revenue
                  FROM
                      (SELECT f.name AS facility_name, 
                              SUM(CASE 
                                    WHEN b.memid <> 0 THEN (f.membercost * b.slots)
                                    WHEN b.memid = 0 THEN (f.guestcost * b.slots)
                              END) AS revenue
                  FROM Bookings AS b
                  LEFT JOIN Facilities AS f
                  ON b.facid = f.facid
                  GROUP BY f.name) AS sub
                  WHERE sub.revenue < 1000
                  ORDER BY sub.revenue;"""
       
    q11 = "Q11. Produce a report of members and who recommended them in alphabetic surname, firstname order"
    
    
    query_11 = """
        SELECT m2.surname AS mem_last_name, m2.firstname AS mem_first_name, 
               m1.surname AS rec_by_last_name, m1.firstname AS rec_by_first_name
        FROM Members AS m1
        INNER JOIN Members as m2
        WHERE m2.recommendedby = m1.memid AND
        m1.surname <> 'GUEST'
        ORDER BY m2.surname, m2.firstname;
        """
    q12 = "Q12. Find the facilities with their usage by member, but not guests"
    
    
    query_12 = """SELECT f.name AS Facility_Name, SUM(b.slots) AS Used_Slots
                   FROM Bookings AS b
                   LEFT JOIN Facilities AS f
                   ON b.facid = f.facid
                   WHERE b.memid <> 0
                   GROUP BY Facility_Name
                   ORDER BY Used_Slots DESC;"""
    
    q13 = "Q13. Find the facilities usage by month, but not guests"
    
    query_13 =  """SELECT EXTRACT(MONTH FROM b.starttime) AS Month, f.name AS Facility_Name, SUM(b.slots) AS Used_Slots
                    FROM Bookings AS b
                    LEFT JOIN Facilities AS f
                    ON b.facid = f.facid
                    WHERE b.memid <> 0
                    GROUP BY Month, Facility_Name
                    ORDER BY Month, Facility_Name;"""
 
    # create a database connection
    conn = create_connection(database)
    with conn: 
        
        print("2. Query all tasks")
        print()
        select_all_tasks(conn, q10, query_10)
        select_all_tasks(conn, q11, query_11)
        select_all_tasks(conn, q12, query_12)
        select_all_tasks(conn, q13, query_13)
        
if __name__ == '__main__':
    main()

2.6.0
2. Query all tasks

Q10. Produce a list of facilities with a total revenue less than 1000.
           The output of facility name and total revenue, sorted by revenue. Remember
           that there's a different cost for guests and members!

SELECT sub.facility_name, sub.revenue
                  FROM
                      (SELECT f.name AS facility_name, 
                              SUM(CASE 
                                    WHEN b.memid <> 0 THEN (f.membercost * b.slots)
                                    WHEN b.memid = 0 THEN (f.guestcost * b.slots)
                              END) AS revenue
                  FROM Bookings AS b
                  LEFT JOIN Facilities AS f
                  ON b.facid = f.facid
                  GROUP BY f.name) AS sub
                  WHERE sub.revenue < 1000
                  ORDER BY sub.revenue;

   facility_name  revenue
0   Table Tennis      180
1  Snooker Table      240
2     Pool Table      270

Q11. Produce a report of member

OperationalError: near "FROM": syntax error